# First dataset

In [1]:
import pandas as pd
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tseh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
CVs = pd.read_csv("C:\\Users\\Tseh\\Downloads\\resumes.csv", delimiter = ';', nrows=20000)
vacancies = pd.read_csv("C:\\Users\\Tseh\\Downloads\\vacancies.csv", nrows=20000)

In [55]:
CVs.shape

(44744, 12)

In [56]:
vacancies.shape

(48564, 15)

In [58]:
CVs = CVs.drop(columns = ['Пол, возраст', 'ЗП', 'Обновление резюме', 'Авто'])
CVs['resume'] = CVs.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
CVs = CVs[['resume']]
CVs.insert(0, 'id', range(1, len(CVs) + 1))
CVs.head()

,id,resume
0,1,Системный администратор Советск (Калининградск...
1,2,"Технический писатель Королев , не готов к пере..."
2,3,"Оператор Тверь , не готова к переезду , не гот..."
3,4,Веб-разработчик (HTML / CSS / JS / PHP / базы ...
4,5,"Региональный менеджер по продажам Москва , не ..."


In [59]:
vacancies = vacancies.drop(columns = ['Ids', 'Employer', 'Salary', 'From', 'To', 'Published at', 'Professional roles', 'Specializations', 'Profarea names'])
vacancies['vacancies'] = vacancies.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
vacancies = vacancies[['vacancies']]
vacancies.insert(0, 'id', range(1, len(vacancies) + 1))
vacancies.head()

,id,vacancies
0,1,Golang Developer (Кипр) От 3 до 6 лет Полный д...
1,2,Е-mail маркетолог От 1 года до 3 лет Полный де...
2,3,Оператор call-центра (удаленно) От 1 года до 3...
3,4,Ведущий SMM специалист От 1 года до 3 лет Полн...
4,5,UX/UI Designer От 1 года до 3 лет Полный день ...


In [60]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(str(text).lower())   

    # stop words  
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [61]:
CVs['Processed_Resume'] = CVs['resume'].apply(preprocess_text)
vacancies['Processed_Description'] = vacancies['vacancies'].apply(preprocess_text)

In [62]:
# Vectorization
tfidf = TfidfVectorizer()
tfidf_matrix_cv = tfidf.fit_transform(CVs['Processed_Resume'])
tfidf_matrix_vacancies = tfidf.transform(vacancies['Processed_Description'])

In [63]:
# cosine similarity 
similarity_matrix = cosine_similarity(tfidf_matrix_cv, tfidf_matrix_vacancies)

matched_indices = similarity_matrix.argmax(axis=1)
CVs['Matched_Vacancy_ID'] = matched_indices

In [64]:
CVs['Matched_Vacancy_Description'] = vacancies.loc[matched_indices, 'vacancies'].values

In [65]:
# new matched dataset   
matched_data = pd.merge(CVs, vacancies, left_on='Matched_Vacancy_ID', right_index=True, suffixes=('_Resume', '_Vacancy'))

In [66]:
matched_data_final = matched_data[['Processed_Resume', 'Processed_Description']]

In [67]:
matched_data_final.head(50)

,Processed_Resume,Processed_Description
0,системный администратор советск калининградска...,системный администратор года лет полный день п...
439,системный администратор омск готов переезду го...,системный администратор года лет полный день п...
1800,системный администратор пенза готов переезду г...,системный администратор года лет полный день п...
2193,системный администратор астана готов переезду ...,системный администратор года лет полный день п...
7135,системный администратор дзержинск нижегородска...,системный администратор года лет полный день п...
8937,системный администратор помощник системного ад...,системный администратор года лет полный день п...
9012,системный администратор уфа готов переезду гот...,системный администратор года лет полный день п...
12827,системный администратор уфа готов переезду гот...,системный администратор года лет полный день п...
13362,системный администратор готов переезду готов к...,системный администратор года лет полный день п...
18742,системный администратор волжский готов переезд...,системный администратор года лет полный день п...
